In [ ]:
import pandas as pd
#import numpy as np
from IPython.display import HTML

#importing plotly and cufflinks in offline mode
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks
import plotly.offline as pyo
from plotly.offline import plot, iplot, init_notebook_mode
pyo.init_notebook_mode()
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='white')

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
#Read CBC output file
df = pd.read_csv(r'C:\Users\abhis\Desktop\new model file\res\cbcoutput.txt', sep='\t')

In [ ]:
df.columns = ['temp']
df[['temp','value']] = df['temp'].str.split(')', expand=True)
df = df.applymap(lambda x: x.strip() if isinstance(x,str) else x)
df[['temp','parameter']] = df['temp'].str.split(' ', expand=True)
df[['parameter','id']] = df['parameter'].str.split('(', expand=True)
df['value'] = df['value'].str.replace(' 0','')
df = df.drop('temp', axis=1)
df = df[~df['value'].str.contains('e-')]

In [ ]:
params = df.parameter.unique()
all_params = {}
cols = {'NewCapacity':['r','t','y'],
            'AccumulatedNewCapacity':['r','t','y'], 
            'TotalCapacityAnnual':['r','t','y'],
            'CapitalInvestment':['r','t','y'],
            'AnnualVariableOperatingCost':['r','t','y'],
            'AnnualFixedOperatingCost':['r','t','y'],
            'SalvageValue':['r','t','y'],
            'DiscountedSalvageValue':['r','t','y'],
            'TotalTechnologyAnnualActivity':['r','t','y'],
            'RateOfActivity':['r','l','t','m','y'],
            'RateOfTotalActivity':['r','t','l','y'],
            'Demand':['r','l','f','y'],
            'TotalAnnualTechnologyActivityByMode':['r','t','m','y'],
            'TotalTechnologyModelPeriodActivity':['r','t'],
            'ProductionByTechnologyAnnual':['r','t','f','y'],
            'AnnualTechnologyEmissionByMode':['r','t','e','m','y'],
            'AnnualTechnologyEmission':['r','t','e','y'],
            'AnnualEmissions':['r','e','y']}

for each in params:
    df_p = df[df.parameter == each]
    df_p[cols[each]] = df_p['id'].str.split(',',expand=True)
    cols[each].append('value')
    df_p = df_p[cols[each]] # Reorder dataframe to include 'value' as last column
    all_params[each] = pd.DataFrame(df_p) # Create a dataframe for each parameter
    df_p = df_p.rename(columns={'value':each})
    #df_p.to_csv(str(each) + '.csv', index=None) # Print data for each paramter to a CSV file

In [ ]:
# List of columns for detailed energy tables and figures
years = pd.Series(range(2015,2031))

det_col = {'BIO':'Biomass',
           'COA':'Coal',
           'CRU':'Crude oil',
           'DSL':'Diesel',
           'ELC':'Electricity',
           'GEO':'Geothermal',
           'GSL':'Gasoline',
           'HFO':'HFO',
           'HYD':'Hydro',
           'JFL':'Jet fuel',
           'LNG':'LNG',
           'LPG':'LPG',
           'PCK':'Petroleum coke',
           'NGS':'Natural gas',
           'OHC':'Other hydrocarbons',
           'PET':'Petroleum products',
           'SOL':'Solar',
           'URN':'Nuclear',
           'WAS':'MSW',
           'WAT':'Water',
           'LND':'Land',
           'WND':'Wind',
           'CP1':'Coffee',
           'CP2':'Cotton',
           'CP3':'Peanuts',
           'CP4':'Maize',
           'CP5':'Rice',
           'CP6':'Soybean',
           'CP7':'Sugarcane',
           'CP8':'Tea',
           'AGR':'Agriculture',
           'BAR':'Barren land',
           'FOR': 'Forests',
           'GRS': 'Grassland & woodland',
           'SET':'Built-up land',
           'OTH':'Other agricultural land',
           'PWR':'Power sector',
           'PUB':'Public supply',
           'EVT':'Evapotranspiration',
           'PRC':'Precipitation',
           'GRC':'Groundwater recharge',
           'SUR':'Surface water'}

# List of columns for aggregated energy tables and figures
agg_col = {'Coal':['Coal'],
           'Oil': ['Diesel','HFO','JFL','Crude oil','Petroleum coke'],
           'Gas': ['Natural gas','LNG','LPG'],
           'Hydro': ['Hydro'],
           'Nuclear': ['Nuclear'],
           'Other renewables': ['Biomass','Geothermal','Solar','MSW','Wind']}

color_dict = {'Biomass':'forestgreen',
              'Coal':'black',
              'Crude oil':'red',
              'Diesel':'firebrick',
              'Electricity':'grey',
              'Geothermal':'purple',
              'Gasoline':'orange',
              'HFO':'brown',
              'Hydro':'blue',
              'Jet fuel':'firebrick',
              'LNG':'purple',
              'LPG':'gold',
              'Natural gas':'orange',
              'Other hydrocarbons':'grey',
              'Petroleum products':'red',
              'Petroleum coke':'red',
              'Solar':'gold',
              'Nuclear':'grey',
              'MSW':'green',
              'Wind':'violet',
              'Oil':'firebrick',
              'Gas':'orange',
              'Other renewables':'green',
              'Coffee':'brown',
              'Cotton':'grey',
              'Peanuts':'orange',
              'Maize':'gold',
              'Rice':'green',
              'Soybean':'firebrick',
              'Sugarcane':'purple',
              'Tea':'pink',
              'Agriculture':'brown',
              'Barren land':'grey',
              'Forests':'forestgreen',
              'Grassland & woodland':'darkgreen',
              'Built-up land':'black',
              'Other agricultural land':'gold',
              'Water':'blue',
              'Power sector':'red',
              'Public supply':'grey',
              'Irrigation':'brown',
              'Evapotranspiration':'green',
              'Precipitation':'blue',
              'Groundwater recharge':'brown',
              'Surface water':'red'}

In [ ]:
def df_filter(df,lb,ub,t_exclude):
    df['t'] = df['t'].str[lb:ub]
    df['value'] = df['value'].astype('float64')
    df = df[~df['t'].isin(t_exclude)].pivot_table(index='y', 
                                          columns='t',
                                          values='value', 
                                          aggfunc='sum').reset_index().fillna(0)
    df = df.reindex(sorted(df.columns), axis=1).set_index('y').reset_index().rename(columns=det_col)
    df['y'] = years
    return df

def df_plot(df,y_title,p_title):
    return df.iplot(x='y',
             kind='bar', 
             barmode='stack',
             xTitle='Year',
             yTitle=y_title,
             color=[color_dict[x] for x in df.columns if x != 'y'],
             title=p_title)

In [ ]:
# Land use

regions = {'C':'Center',
           'N':'North',
           'S':'South'}

crops = {'CP1':'Coffee',
         'CP2':'Cotton',
         'CP3':'Peanuts',
         'CP4':'Maize',
         'CP5':'Rice',
         'CP6':'Soybean',
         'CP7':'Sugarcane',
         'CP8':'Tea'}

water_supply = {'I':'Irrigated',
                'R':'Rain-fed'}

input_level = {'L':'Low',
               'I':'Intermediate',
               'H':'High'}

In [ ]:
crops_total_df = all_params['TotalTechnologyAnnualActivity'][all_params['TotalTechnologyAnnualActivity'].t.str.startswith('LNDCP')].drop('r', axis=1)
crops_total_df = df_filter(crops_total_df,3,6,[])
df_plot(crops_total_df,'Land area (1000 sq.km.)','Area by crop')

In [ ]:
land_total_df = all_params['TotalTechnologyAnnualActivity'][all_params['TotalTechnologyAnnualActivity'].t.str.startswith('LND')].drop('r', axis=1)
land_total_df = df_filter(land_total_df,3,6,['AGR'])
land_total_df['Agriculture'] = 0
for each in ['Coffee','Cotton','Peanuts','Maize','Rice','Soybean','Sugarcane','Tea']:
    if each in land_total_df:
        land_total_df['Agriculture'] = land_total_df['Agriculture'] + land_total_df[each]
        land_total_df = land_total_df.drop(each,axis=1)
df_plot(land_total_df,'Land area (1000 sq.km.)','Area by land cover type')

In [ ]:
for each in regions.keys():
    crops_region_df = all_params['TotalTechnologyAnnualActivity'][all_params['TotalTechnologyAnnualActivity'].t.str.startswith('LNDCP')].drop('r', axis=1)
    crops_region_df = crops_region_df[crops_region_df.t.str[6:7] == each]
    crops_region_df = df_filter(crops_region_df,3,6,[])
    df_plot(crops_region_df,'Land area (1000 sq.km.)','Area by crop (' + regions[each] + ' region)')

In [ ]:
for each in regions.keys():
    land_region_df = all_params['TotalTechnologyAnnualActivity'][all_params['TotalTechnologyAnnualActivity'].t.str.startswith('LND')].drop('r', axis=1)
    land_region_df = land_region_df[land_region_df.t.str[6:7] == each]
    land_region_df = df_filter(land_region_df,3,6,['AGR'])
    land_region_df['Agriculture'] = 0
    for crop in ['Coffee','Cotton','Peanuts','Maize','Rice','Soybean','Sugarcane','Tea']:
        if crop in land_region_df:
            land_region_df['Agriculture'] = land_region_df['Agriculture'] + land_region_df[crop]
            land_region_df = land_region_df.drop(crop,axis=1)
    df_plot(land_region_df,'Land area (1000 sq.km.)','Area by land cover type (' + regions[each] + ' region)')

In [ ]:
for each in water_supply.keys():
    crops_ws_df = all_params['TotalTechnologyAnnualActivity'][all_params['TotalTechnologyAnnualActivity'].t.str.startswith('LNDCP')].drop('r', axis=1)
    crops_ws_df = crops_ws_df[crops_ws_df.t.str[8:9] == each]
    crops_ws_df = df_filter(crops_ws_df,3,6,[])
    df_plot(crops_ws_df,'Land area (1000 sq.km.)','Area by crop (' + water_supply[each] + ')')

In [ ]:
crops_prod_df = all_params['Demand'][all_params['Demand'].f.str.startswith('CRP')].drop('r', axis=1)
crops_prod_df['f'] = crops_prod_df['f'].str[3:6]
crops_prod_df['value'] = crops_prod_df['value'].astype('float64').div(10)

crops_prod_df = crops_prod_df.pivot_table(index='y', 
                                          columns='f',
                                          values='value',
                                          aggfunc='sum').reset_index().fillna(0)
crops_prod_df = crops_prod_df.reindex(sorted(crops_prod_df.columns), axis=1).set_index('y').reset_index().rename(columns=det_col)
crops_prod_df['y'] = years
df_plot(crops_prod_df,'Production (Million tonnes)','Crop production')

In [ ]:
crops_yield_df = crops_prod_df*10/crops_total_df
crops_yield_df['y'] = years

crops_yield_df.iplot(x='y',
                     mode='lines+markers', 
                     xTitle='Year',
                     yTitle='Yield (t/ha)',
                     size=10,
                     color=[color_dict[x] for x in crops_yield_df.columns if x != 'y'],
                     title='Yield (tonnes/hectare)')